In [1]:
import os
import numpy as np
import pandas as pd
from rdkit import Chem
from openforcefield.topology import Molecule
import timeout_decorator
from tqdm.notebook import tqdm

In [2]:
drugtable = pd.read_csv('drug_table_cleaned.csv')
drugtable.head()

,PDB ID,Generic Name,Brand Name,DrugBank ID,ATC Codes,Ligand ID,Target Name,UniProt ID,PDB ID.1,Seq. Identity
0,1QYX,4-Androstenedione,NaN,DB01536,NaN,ASD,Estradiol 17-beta-dehydrogenase 1,P14061,1QYX,89%
1,3RUK,Abiraterone,Zytiga,DB05812,L02BX03,AER,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093,3RUK,97%
2,4NKV,Abiraterone,Zytiga,DB05812,L02BX03,AER,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093,4NKV,97%
3,4R1Z,Abiraterone,Zytiga,DB05812,L02BX03,AER,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093,4R1Z,50%
4,2QMJ,Acarbose,Acarbose#Glucobay#Precose#Prandase,DB00284,A10BF01#A10BD17,"ACR,QPS","Maltase-glucoamylase, intestinal",O43451,2QMJ,99%


## Extract ligands from pdb files and convert to sdf
We parse the dataframe `drugtable`, load the pdb in entry `PDB ID` and extract the ligands with residue IDs in entry `Ligand ID`. Some residues will not be found in the pdb, because we split the rows above and the residue ID belongs to a different pdb entry. 

In [3]:
@timeout_decorator.timeout(20, timeout_exception=TimeoutError)
def extract_ligands(PDBID, residues):
    if os.path.exists(f'pdb/{PDBID}.pdb'):
        mol = Chem.MolFromPDBFile(f'pdb/{PDBID}.pdb', sanitize=False, removeHs=True)
        if mol is not None:
            lig_dict = Chem.rdmolops.SplitMolByPDBResidues(mol, whiteList=residues)
            for res, lig in lig_dict.items():
#                 lig = Chem.AddHs(lig, addCoords=True)
#                 Chem.SanitizeMol(lig)
#                 #Chem.rdmolops.AssignAtomChiralTagsFromStructure(lig)
#                 Chem.rdmolops.AssignStereochemistry(lig)
                lig.SetProp("PDBID", PDBID)   
                lig_dict[res] = lig
            return lig_dict
        else:
            print(f'Reading {PDBID} failed')
            return {}
    else:
        print(f'File pdb/{PDBID}.pdb does not exist.') 
        return {}

In [4]:
pbar = tqdm(drugtable.iterrows(), total=drugtable.shape[0])
for i, row in pbar:
    PDBID = row["PDB ID"]
    pbar.set_description(f'PDB {PDBID}')
    residues = row["Ligand ID"].split(',')
    try:
        lig_dict = extract_ligands(PDBID, residues)
    except TimeoutError:
        print('Extraction timeout. Continue with next pdb.')
        continue
    for res, lig in lig_dict.items():
        drugtable.loc[i, "Ligand ID"] = res
        for j, prop in row.iteritems():
            lig.SetProp(j, str(prop))
        drugtable.loc[i, "RDMol"] = lig
    if len(lig_dict) > 1:
        print('More than one ligand.')
drugtable.head()

Reading 4BDS failed



,PDB ID,Generic Name,Brand Name,DrugBank ID,ATC Codes,Ligand ID,Target Name,UniProt ID,PDB ID.1,Seq. Identity,RDMol
0,1QYX,4-Androstenedione,NaN,DB01536,NaN,ASD,Estradiol 17-beta-dehydrogenase 1,P14061,1QYX,89%,<rdkit.Chem.rdchem.Mol object at 0x7fa9b93316f0>
1,3RUK,Abiraterone,Zytiga,DB05812,L02BX03,AER,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093,3RUK,97%,<rdkit.Chem.rdchem.Mol object at 0x7fa9b9320f30>
2,4NKV,Abiraterone,Zytiga,DB05812,L02BX03,AER,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093,4NKV,97%,<rdkit.Chem.rdchem.Mol object at 0x7fa9b9331870>
3,4R1Z,Abiraterone,Zytiga,DB05812,L02BX03,AER,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093,4R1Z,50%,<rdkit.Chem.rdchem.Mol object at 0x7fa9b92c9750>
4,2QMJ,Acarbose,Acarbose#Glucobay#Precose#Prandase,DB00284,A10BF01#A10BD17,ACR,"Maltase-glucoamylase, intestinal",O43451,2QMJ,99%,<rdkit.Chem.rdchem.Mol object at 0x7fa9b92c93f0>


# Split molecules and remove all rows where RDMol is NaN
Some PDBs contain several copies of the druglike molecules, which are so far stored in one molecule object. In the following, we split these up. 

In [5]:
extended_drugtable = pd.DataFrame()
pbar = tqdm(drugtable.iterrows(), total=drugtable.shape[0])
for i, row in pbar:
    PDBID = row["PDB ID"]
    pbar.set_description(f'PDB {PDBID}')
    mol_to_split = row['RDMol']
    if not pd.isna(mol_to_split):
        mols = Chem.GetMolFrags(mol_to_split, asMols=True)
        for mol in mols:
            new_row = row.copy()
            new_row['RDMol'] = mol
            extended_drugtable = extended_drugtable.append(new_row)
extended_drugtable.reset_index(drop=True, inplace=True)
extended_drugtable.head()

,ATC Codes,Brand Name,DrugBank ID,Generic Name,Ligand ID,PDB ID,PDB ID.1,RDMol,Seq. Identity,Target Name,UniProt ID
0,NaN,NaN,DB01536,4-Androstenedione,ASD,1QYX,1QYX,<rdkit.Chem.rdchem.Mol object at 0x7fa9b92f6f30>,89%,Estradiol 17-beta-dehydrogenase 1,P14061
1,L02BX03,Zytiga,DB05812,Abiraterone,AER,3RUK,3RUK,<rdkit.Chem.rdchem.Mol object at 0x7fa9b92bee70>,97%,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093
2,L02BX03,Zytiga,DB05812,Abiraterone,AER,3RUK,3RUK,<rdkit.Chem.rdchem.Mol object at 0x7fa9b92fc450>,97%,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093
3,L02BX03,Zytiga,DB05812,Abiraterone,AER,3RUK,3RUK,<rdkit.Chem.rdchem.Mol object at 0x7fa9b92fc3f0>,97%,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093
4,L02BX03,Zytiga,DB05812,Abiraterone,AER,3RUK,3RUK,<rdkit.Chem.rdchem.Mol object at 0x7fa9b92fc150>,97%,"Steroid 17-alpha-hydroxylase/17,20 lyase",P05093


## Assign bond orders from reference structures of the DrugBank
The reference structures are publically available here: drugbank.com/releases/latest#open-data (retrieved on Nov 25, 2020).

In [6]:
mol_supplier = Chem.SDMolSupplier('../open structures.sdf', sanitize=False)
drugbank_dict = {}
for mol in mol_supplier:
    if mol is not None:
        drugbank_dict[mol.GetProp('DRUGBANK_ID')] = mol

In [7]:
from rdkit.Chem import AllChem
pbar = tqdm(extended_drugtable.iterrows(), total=extended_drugtable.shape[0])
for i, row in pbar:
    drug_bank_id = row['DrugBank ID']
    pbar.set_description(drug_bank_id)
    mol_wo_bond_orders = row['RDMol']
    mol_wo_bond_orders_wo_Hs = Chem.RemoveAllHs(row['RDMol'])
    if drug_bank_id in drugbank_dict:
        template = Chem.RemoveAllHs(drugbank_dict[drug_bank_id])
        try:
            mol = AllChem.AssignBondOrdersFromTemplate(template, mol_wo_bond_orders_wo_Hs)
            mol_H = AllChem.AddHs(mol, addCoords=True)
            Chem.SanitizeMol(mol_H)
            Chem.AssignStereochemistryFrom3D(mol_H)
            extended_drugtable.loc[i, 'RDMol'] = mol_H
        except:
            print(f"Issue with PDB {row['PDB ID']} (DB ID {row['DrugBank ID']})")
            extended_drugtable.loc[i, 'RDMol'] = np.nan            
            ## possible solution
            #mol_wo_bond_orders_wo_Hs = Chem.RemoveAllHs(drugtable.loc[drugtable['PDB ID']==row['PDB ID']]['RDMol'].iloc[0])
            #mol = AllChem.AssignBondOrdersFromTemplate(template, mol_wo_bond_orders_wo_Hs)    
    else:
        print(f'DrugBank ID {drug_bank_id} not available')
        extended_drugtable.loc[i, 'RDMol'] = np.nan        

Issue with PDB 5NN8 (DB ID DB00284)
Issue with PDB 3WEO (DB ID DB00284)


RDKit WARNING: [00:41:40] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:40] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:40] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:40] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:40] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:40] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:40] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:40] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:40] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:40] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:40] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:40] WARNING: More than one match

Issue with PDB 1ITU (DB ID DB01597)
Issue with PDB 1ITU (DB ID DB01597)


RDKit WARNING: [00:41:41] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:41] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:41] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:41] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:41] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:41] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:41] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:41] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:41] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:41] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:41] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:41] WARNING: More than one match

Issue with PDB 2TOD (DB ID DB06243)
Issue with PDB 2TOD (DB ID DB06243)
Issue with PDB 2TOD (DB ID DB06243)
Issue with PDB 2TOD (DB ID DB06243)


...

RDKit WARNING: [00:41:42] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:42] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:42] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:42] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:42] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:42] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:42] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:42] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:42] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:43] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:43] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:43] WARNING: More than one match

Issue with PDB 3G0E (DB ID DB01268)
Issue with PDB 3G0F (DB ID DB01268)
Issue with PDB 3G0F (DB ID DB01268)
Issue with PDB 1T03 (DB ID DB00300)


RDKit WARNING: [00:41:43] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:43] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:43] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:43] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:43] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:43] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:43] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:43] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:43] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:43] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:43] WARNING: More than one matching pattern found - picking one
RDKit WARNING: [00:41:43] WARNING: More than one match

DrugBank ID DB01361 not available
Issue with PDB 3C0Z (DB ID DB02546)
Issue with PDB 3C0Z (DB ID DB02546)
Issue with PDB 3C0Z (DB ID DB02546)
Issue with PDB 1A4G (DB ID DB00558)



RDKit ERROR: [00:41:43] WARNING: More than one matching pattern found - picking on[00:41:43] Explicit valence for atom # 11 C, 5, is greater than permitted


## Assign a unique identifier to all conformers (based on DrugBank ID) and write to SDF file
The identifier has the format `PDB_DBXXXXX_YY`, where `XXXXX` is part of the DrugBank ID and `YY` is a conformer index starting at 99 and decreasing.

In [8]:
extended_drugtable = extended_drugtable[np.invert(extended_drugtable['RDMol'].isna())]

In [9]:
pbar = tqdm(extended_drugtable['DrugBank ID'].unique(), total=extended_drugtable['DrugBank ID'].unique().shape[0])
for drug_bank_id in pbar:
    for i, (index, row) in enumerate(extended_drugtable[extended_drugtable['DrugBank ID']==drug_bank_id].iterrows()):
        identifier = f'PDB_{drug_bank_id}_{99-i:02d}'
        pbar.set_description(f'Conformer {identifier}')
        extended_drugtable.loc[index, 'identifier'] = identifier
        extended_drugtable.loc[index, 'RDMol'].SetProp('_Name', identifier)

## Write molecules to SDF

In [10]:
os.makedirs('sdf_crystal', exist_ok=True)
pbar = tqdm(extended_drugtable.iterrows(), total=extended_drugtable.shape[0])
for i, row in pbar:
    identifier = row['identifier']
    pbar.set_description(f'Conformer {identifier}')
    off_mol = Molecule.from_rdkit(row['RDMol'], allow_undefined_stereo=True)
    off_mol.to_file(f'sdf_crystal/{identifier}.sdf', 'SDF') 

Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
Warning (not error because allow_undefined_stereo=True): 
